In [ ]:
import xml.etree.cElementTree as ET
import pprint
import os
import re
import json
from pymongo import MongoClient

def get_db():
    # For local use
    from pymongo import MongoClient
    client = MongoClient('localhost:27017')
    # 'examples' here is the database name. It will be created if it does not exist.
    db = client.examples
    return db

#This function counts the tags that are present in the data file. The output is the dictionary tagcount which will
#hold the tags as keys and the number of times they appear in the file as entries.
'''BEGIN FUNCTION TO COUNT TAGS'''
def counttags(fname):
    tagcount = {}
    with open(fname,'r') as f:
        for event,elem in ET.iterparse(f, events=('start',)):
            el = shape_element(elem)
            '''
            if elem.tag in tagcount:
                tagcount[elem.tag] += 1
            elif elem.tag not in tagcount:
                tagcount[elem.tag] = 1
            '''
    return tagcount
'''END FUNCTION TO COUNT TAGS'''

'''BEGIN BLOCK TO AUDIT STREET NAMES'''
def is_street_name(elem):
    return (elem.attrib[k] == 'addr:street')

def audit(fname,tagcount):
    with open(fname,'r') as f:
        for event,elem in ET.iterparse(f, events=('start',)):
            if elem.tag == 'way':
                for tag in elem.iter('tag'):
                    if is_street_name(tag):
                        pass
'''END BLOCK TO AUDIT STREET NAMES'''   

'''BEGIN BLOCK TO ORGANIZE DATA INTO DESIRED STRUCTURE'''
def datadesign(fname):
    data = []
    with open(fname,'r') as f:
        for event,elem in ET.iterparse(f, events=('start',)):
            el = shape_element(elem)
            data.append(el)
            f.close()
    return data
    
            
def shape_element(element):
    node = {}
    if element.tag == 'node' or element.tag == 'way':
        node['type'] = element.tag
        #Declare empty dicts and array for node info, address, and coordinates.
        createdinfo = {}
        addressinfo = {}
        posinfo = []
        ndrefinfo = []
        #Loop through element to collect all data and add it to the node dictionary
        for e in element.attrib:
            #test stuff here
            #print element.keys()
            
            
            '''BEGIN INFO FOR MAIN NODE'''
            if e == 'id':
                node[e] = element.get(e)
            if e == 'visible':
                node[e] = element.get(e)
            #info to go into created and pos dict/array
            if e == 'version' or e == 'changeset' or e == 'timestamp' or e == 'user' or e == 'uid':
                createdinfo[e] = element.get(e)
            if e == 'lat' or e == 'lon':
                posinfo.append(element.get(e))
            '''END INFO FOR MAIN NODE'''  
            
            '''BEGIN INFO FOR SECOND LEVEL TAGS''' 
            for subtags in element.findall("./tag"):
                '''ADDRESS SECTION'''
                if subtags.get('k')[0:4] == 'addr':
                    formattag = subtags.get('k')[5:]
                    addressinfo[formattag] = subtags.get('v')
                    #print formattag
                else:
                    node[subtags.get('k')] = subtags.get('v')
            '''END INFO FOR SECOND LEVEL TAGS'''
            
            '''SPECIAL FOR WAY'''
            if node['type'] == 'way':
                for subtags in element.findall("./nd"):
                    ndrefinfo.append(subtags.get('ref'))
                  
                node['node_refs'] = ndrefinfo
            
            #Final appending
            node['created'] = createdinfo
            node['pos'] = posinfo
            node['address'] = addressinfo
            
        
        return node
    else:
        return None
        
'''END BLOCK TO ORGANIZE DATA INTO DESIRED STRUCTURE'''

'''BEGIN BLOCK TO REMOVE PROBLEMATIC CHARACTERS'''
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')




'''END BLOCK TO REMOVE PROBLEM CHARACTERS'''
        
    
'''BEGIN MAIN FUNCTION'''
def main():
    filename = 'sample'
    db = get_db()
    #tagcount = counttags(filename)
    #audit(filename,tagcount)
    data = datadesign(filename)
    with open('saver.txt','w') as saver:
        saver.write(data)
    print data

main()

